### 📋 HWP RAG 시스템 구축 프로젝트

이 노트북에서는 HWP(한글) 문서를 PDF로 변환하고, AI 기반 문서 분석을 통해 RAG(Retrieval-Augmented Generation) 시스템을 구축하는 전체 과정을 다룹니다.

#### 🔧 필수 패키지 설치
HWP 파일을 처리하기 위해 필요한 `olefile` 패키지를 설치합니다.


In [ ]:
%pip install olefile

#### 📄 HWP → PDF 일괄 변환
Windows COM 객체를 사용하여 HWP 파일들을 PDF 형식으로 일괄 변환합니다. 한글 프로그램이 설치된 Windows 환경에서만 작동합니다.


In [ ]:
from pathlib import Path
import win32com.client as win32

def hwp_to_pdf_batch(folder: str, out_dir: str):
    folder = Path(folder).resolve()
    out_dir = Path(out_dir).resolve()
    out_dir.mkdir(parents=True, exist_ok=True)

    hwp = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")
    hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

    for hwp_file in folder.glob("*.hwp"):
        try:
            hwp.Open(str(hwp_file))
        except Exception as e:
            print("✖ 열기 실패:", hwp_file.name, e)
            continue

        pdf_path = out_dir / f"{hwp_file.stem}.pdf"
        hwp.SaveAs(str(pdf_path), "PDF", "")
        hwp.Run("FileClose")          # ← 핵심 변경: 문서 닫기
        print("✔", pdf_path.name)

    hwp.Quit()

hwp_to_pdf_batch("./data", "./data/pdf")

#### 🤖 Docling을 이용한 고급 PDF 처리
Docling 라이브러리를 사용하여 PDF 문서를 마크다운으로 변환하고, 이미지와 표를 추출합니다. 고품질 문서 파싱과 구조화된 데이터 추출이 가능합니다.


In [1]:
import logging
import time
from pathlib import Path
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

# 로깅 설정 - INFO 레벨로 변환 과정을 모니터링
logging.basicConfig(level=logging.INFO)
_log = logging.getLogger(__name__)

# 입력 PDF 파일 경로와 출력 디렉토리 설정
input_doc_path = Path("data/pdf/proposal_request_2025_ai_economic_policy_service.pdf")
output_dir = Path("data/pdf/output")

# PDF 파이프라인 옵션 구성
pipeline_options = PdfPipelineOptions()
pipeline_options.images_scale = 2.0  # 이미지 해상도 스케일 설정
pipeline_options.generate_picture_images = True

# 문서 변환기 초기화 - PDF 형식 옵션과 함께 설정
doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

# 변환 시작 시간 기록
start_time = time.time()

# PDF 문서 변환 실행
conv_res = doc_converter.convert(input_doc_path)

INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash 4bba35b4916b4b6fae4b5936f7e9e6a2
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered picture descriptions: ['vlm', 'api']
INFO:docling.pipeline.base_pipeline:Processing document proposal_request_2025_ai_economic_policy_service.pdf
c:\Users\user\anaconda3\envs\test_venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a

KeyboardInterrupt: 

#### 💾 마크다운 파일 저장 및 변환 완료
변환된 문서를 마크다운 형식으로 저장하고 이미지 참조를 포함한 파일을 생성합니다.


In [ ]:
from itertools import accumulate

# 출력 디렉토리 생성 (부모 디렉토리도 함께 생성)
output_dir.mkdir(parents=True, exist_ok=True)
doc_filename = conv_res.input.file.stem

# 이미지 참조가 포함된 전체 마크다운 파일 저장
md_filename_with_refs = output_dir / f"{doc_filename}-with-image-refs.md"
conv_res.document.save_as_markdown(md_filename_with_refs, image_mode=ImageRefMode.REFERENCED)

# 변환 완료 시간 계산
end_time = time.time() - start_time

# 변환 완료 로그 출력
_log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")

#### 🖼️ AI 기반 이미지 설명 생성
Gemini 2.5 Flash 모델을 사용하여 추출된 이미지들에 대한 자동 캡션을 생성하고, 마크다운 파일의 이미지 참조를 텍스트 설명으로 교체합니다.


In [ ]:
import base64
import glob
import re
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20")

# 이미지 폴더에서 PNG 파일들 가져오기
image_folder = "data/pdf/output/proposal_request_2025_ai_economic_policy_service-with-image-refs_artifacts/"
image_paths = glob.glob(f"{image_folder}*.png")

# 배치 처리를 위한 메시지 리스트 생성
messages_batch = []
image_filenames = []

for image_path in image_paths:
    with open(image_path, "rb") as f:
        encoded_image = base64.b64encode(f.read()).decode("utf-8")
    
    message = HumanMessage(content=[
        {"type": "text", "text": "이 이미지를 한글로 간단하고 명확하게 설명해주세요. 표나 차트의 경우 주요 내용을 요약해주세요."},
        {"type": "image_url", "image_url": f"data:image/png;base64,{encoded_image}"}
    ])
    
    messages_batch.append([message])
    image_filenames.append(Path(image_path).name)

# 배치 처리로 모든 이미지 설명 생성
print("이미지 설명 생성 중...")
batch_results = llm.batch(messages_batch)

# 이미지 파일명과 설명을 매핑
image_descriptions = {}
for filename, result in zip(image_filenames, batch_results):
    image_descriptions[filename] = result.content
    print(f"{filename}: {result.content}\n")

# 마크다운 파일 읽기
with open(md_filename_with_refs, 'r', encoding='utf-8') as f:
    markdown_content = f.read()

# 이미지 참조를 캡션으로 교체
def replace_image_with_caption(match):
    full_path = match.group(1)  # 전체 경로 추출
    image_filename = Path(full_path).name  # 파일명만 추출
    if image_filename in image_descriptions:
        caption = image_descriptions[image_filename]
        return f"\n**[이미지 설명]** {caption}\n"
    return match.group(0)  # 설명이 없으면 원본 유지

# 이미지 참조 패턴 찾기 및 교체 (백슬래시와 슬래시 모두 처리)
image_pattern = r'!\[.*?\]\(([^)]+\.png)\)'
updated_markdown = re.sub(image_pattern, replace_image_with_caption, markdown_content)

# 캡션이 포함된 마크다운 파일 저장
md_filename_with_captions = output_dir / f"{doc_filename}-with-captions.md"
with open(md_filename_with_captions, 'w', encoding='utf-8') as f:
    f.write(updated_markdown)

print(f"캡션이 포함된 마크다운 파일이 저장되었습니다: {md_filename_with_captions}")

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=500,
    separators=["##\n","\n\n", "\n", " ", ""]
)

# 캡션이 적용된 파일 내용 읽기
with open(md_filename_with_captions, 'r', encoding='utf-8') as f:
    captioned_content = f.read()

# Document 객체로 생성
document = Document(page_content=captioned_content, metadata={"source": str(md_filename_with_captions)})

# split_documents를 사용하여 Document 객체들로 분할
splits = text_splitter.split_documents([document])

# splits 길이 분포 파악
split_lengths = [len(split.page_content) for split in splits]
print(f"총 분할된 청크 수: {len(splits)}")
print(f"평균 청크 길이: {sum(split_lengths) / len(split_lengths):.1f} 문자")
print(f"최소 청크 길이: {min(split_lengths)} 문자")
print(f"최대 청크 길이: {max(split_lengths)} 문자")

#### 🗄️ Qdrant 벡터 데이터베이스 설정
하이브리드 검색을 위해 Dense 임베딩(BGE-M3)과 Sparse 임베딩(BM25)을 함께 사용하는 Qdrant 벡터 스토어를 구성합니다.


In [1]:
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams

# Qdrant 클라이언트 설정
client = QdrantClient(host="localhost", port=6333)

dense_embeddings = OllamaEmbeddings(model="bge-m3")
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

# 컬렉션 이름 설정
collection_name = f"hwp_rag"
try:
    client.create_collection(
        collection_name=collection_name,
        vectors_config={"dense": VectorParams(size=1024, distance=Distance.COSINE)},
        sparse_vectors_config={
            "sparse": SparseVectorParams(index=models.SparseIndexParams(on_disk=False))
        },
    )
    print(f"새 컬렉션 '{collection_name}' 생성됨")
    qdrant = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=dense_embeddings,
        sparse_embedding=sparse_embeddings,
        retrieval_mode=RetrievalMode.HYBRID,
        vector_name="dense",
        sparse_vector_name="sparse",
    )

    # 문서를 벡터 스토어에 추가
    print("문서를 벡터 스토어에 추가 중...")
    qdrant.add_documents(splits)
    print(f"✅ {len(splits)}개의 문서가 Qdrant에 저장되었습니다.")
    
except:
    qdrant = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=dense_embeddings,
        sparse_embedding=sparse_embeddings,
        retrieval_mode=RetrievalMode.HYBRID,
        vector_name="dense",
        sparse_vector_name="sparse",
    )

#### 🎯 리랭커 기반 검색 품질 향상
BGE-Reranker-v2-M3 모델을 사용하여 초기 검색 결과를 재순위화하고, 가장 관련성 높은 문서들만 선별합니다.


In [2]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

retriever = qdrant.as_retriever(
    search_kwargs={"k": 10}
)

# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=5)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [3]:
def print_search_results(docs, title):
    """검색 결과를 이쁘게 출력하는 함수"""
    print(f"{title}")
    print("=" * 80)
    print(f"검색된 문서 수: {len(docs)}개\n")
    
    for i, doc in enumerate(docs, 1):
        print(f"📄 문서 {i}")
        print(f"📂 출처: {doc.metadata.get('source', 'N/A')}")
        print(f"📃 페이지: {doc.metadata.get(' page', 'N/A')}")
        print(f"📝 내용 미리보기:")
        print(doc.page_content)
        print("-" * 40)


In [ ]:
# 검색 쿼리
query = "AI 기반 경제정책정보 제공 서비스의 구조를 설명해주세요"

# 기본 retriever 검색
basic_docs = retriever.invoke(query)
print_search_results(basic_docs, "🔍 기본 Retriever 검색 결과:")

In [ ]:
# Reranker 적용된 검색
compressed_docs = compression_retriever.invoke(query)
print_search_results(compressed_docs, "🎯 Reranker 적용된 검색 결과:")

In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain_core.documents import Document
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# State 정의
class RAGState(TypedDict):
    question: str
    documents: List[Document]
    answer: str

# LLM 초기화
llm = ChatOllama(
    model="qwen3",
    temperature=0
)

# RAG 프롬프트 템플릿
rag_prompt = ChatPromptTemplate.from_template("""
다음 문서들을 참고하여 질문에 답변해주세요.

문서들:
{context}

질문: {question}

답변은 한국어로 작성하고, 문서의 내용을 바탕으로 정확하고 자세하게 답변해주세요.
답변:
""")

def retrieve_documents(state: RAGState) -> RAGState:
    """문서 검색 단계"""
    print("📚 문서 검색 중...")
    question = state["question"]
    documents = compression_retriever.invoke(question)
    print(f"✅ {len(documents)}개의 문서를 검색했습니다.")
    
    return {
        "question": question,
        "documents": documents,
        "answer": ""
    }

def generate_answer(state: RAGState) -> RAGState:
    """답변 생성 단계"""
    print("🤖 답변 생성 중...")
    question = state["question"]
    documents = state["documents"]
    
    # 문서 내용을 컨텍스트로 결합
    context = "\n\n".join([doc.page_content for doc in documents])
    
    # LLM 체인 구성
    chain = rag_prompt | llm | StrOutputParser()
    
    # 답변 생성
    answer = chain.invoke({
        "context": context,
        "question": question
    })
    
    print("✅ 답변이 생성되었습니다.")
    
    return {
        "question": question,
        "documents": documents,
        "answer": answer
    }

# LangGraph 워크플로우 구성
workflow = StateGraph(RAGState)

# 노드 추가
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("generate", generate_answer)

# 엣지 추가
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# 그래프 컴파일
rag_app = workflow.compile()

In [ ]:
question = "/no_think 입찰가격 평가는 어떻게 이뤄지나요?"
initial_state = {"question": question, "documents": [], "answer": ""}

for chunk, metadata in rag_app.stream(initial_state, stream_mode="messages"):
    if chunk.content:
        print(chunk.content, end="", flush=True)